In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("../data/breast-cancer-wisconsin.txt")

In [3]:
df.head()

,Index,ID,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,0,1241035,7,8,3,7,4,5,7,8,2,4
1,1,1107684,6,10,5,5,4,10,6,10,1,4
2,2,691628,8,6,4,10,10,1,3,5,1,4
3,3,1226612,7,5,6,3,3,8,7,4,1,4
4,4,1142706,5,10,10,10,6,10,6,5,2,4


In [4]:
df.describe()

,Index,ID,Clump Thickness
count,15855.000000,1.585500e+04,15855.000000
mean,7927.000000,1.125972e+06,8.263198
std,4577.088594,9.957879e+05,7.393603
min,0.000000,6.163400e+04,1.000000
25%,3963.500000,1.076352e+06,7.000000
50%,7927.000000,1.111249e+06,8.000000
75%,11890.500000,1.198641e+06,10.000000
max,15854.000000,1.345435e+07,100.000000


In [5]:
good_indices = []
for i in range(len(df.Class)):
    if df.Class[i] == '2' or df.Class[i] == '4':
        good_indices.append(True)
    else:
        good_indices.append(False)

In [6]:
df = df[good_indices]

In [7]:
df.describe()

,Index,ID,Clump Thickness
count,15620.000000,1.562000e+04,15620.000000
mean,7928.076248,1.033424e+06,7.583099
std,4576.334717,2.762936e+05,2.168477
min,0.000000,6.163400e+04,1.000000
25%,3968.750000,1.076352e+06,7.000000
50%,7925.500000,1.111249e+06,8.000000
75%,11890.250000,1.198641e+06,10.000000
max,15854.000000,1.345435e+07,10.000000


In [8]:
#Set classes to 0 or 1
df['Class'] = df['Class'].apply(lambda x: 0 if x == '2' else 1)

In [9]:
df.Class.value_counts()

1    15164
0      456
Name: Class, dtype: int64

In [10]:
df = df.apply(pd.to_numeric, errors='coerce')
df.head()

,Index,ID,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,0,1241035,7,8,3,7,4,5.0,7,8,2,1
1,1,1107684,6,10,5,5,4,10.0,6,10,1,1
2,2,691628,8,6,4,10,10,1.0,3,5,1,1
3,3,1226612,7,5,6,3,3,8.0,7,4,1,1
4,4,1142706,5,10,10,10,6,10.0,6,5,2,1


In [11]:
for i in df.columns:
    if i in ["Index", "ID"]:
        continue
    print(i, set(df[i]))

Clump Thickness {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
Uniformity of Cell Size {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
Uniformity of Cell Shape {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
Marginal Adhesion {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
Single Epithelial Cell Size {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
Bare Nuclei {nan, 1.0, 2.0, 3.0, nan, 5.0, 6.0, nan, 8.0, 9.0, 10.0, 7.0, nan, 4.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan}
Bland Chromatin {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
Normal Nucleoli {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
Mitoses {1, 2, 3, 4, 5, 6, 7, 8, 10}
Class {0, 1}


In [12]:
def clean_col(df, name):
    #good_indices = []
    #print(name)
    #for i in range(len(df[name])):
    #    if df[name][i] >0 or df[name][i] <= 10:
    #        good_indices.append(True)
    #    else:
    #        good_indices.append(False)
    index = [i[0] and i[1] for i in zip(np.array(df[name] > 0), np.array(df[name] <= 10))]
    #Take the mean conditional on the classes
    means = df[index].groupby('Class').mean()[name]
    mean_0 = means[0]
    mean_1 = means[1]
    
    class_0 = df['Class'] == 0
    index_0 = [not i[0] and i[1] for i in zip(index, class_0)]
    df[name][index_0] = mean_0
    
    index_1 = [not i[0] and not i[1] for i in zip(index, class_0)]
    df[name][index_1] = mean_1

In [13]:
for i in df.columns:
    if i == 'Index' or i == 'ID' or i == 'Class':
        continue
    clean_col(df, i)
    print(i, set(df[i]))

Clump Thickness {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
Uniformity of Cell Size {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
Uniformity of Cell Shape {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
Marginal Adhesion {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
Single Epithelial Cell Size {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
Bare Nuclei {1.0, 2.0, 3.0, 1.3484162895927603, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 6.615881809787627, 4.0}
Bland Chromatin {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
Normal Nucleoli {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
Mitoses {1, 2, 3, 4, 5, 6, 7, 8, 10}


/Users/tmcclintock/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/tmcclintock/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [14]:
df.to_pickle("../data/BCC_pickled.pkl")

In [15]:
for c in df.columns:
    if c in ["Index", "ID", "Class"]:
        continue
    m = np.array([df[c], df.Class])
    print(f"{c}--Class cov:")
    print(np.cov(m))
    print("")

Clump Thickness--Class cov:
[[4.70229451 0.13508502]
 [0.13508502 0.02834291]]

Uniformity of Cell Size--Class cov:
[[6.10103678 0.16248966]
 [0.16248966 0.02834291]]

Uniformity of Cell Shape--Class cov:
[[4.54958293 0.12194769]
 [0.12194769 0.02834291]]

Marginal Adhesion--Class cov:
[[8.47902285 0.10696721]
 [0.10696721 0.02834291]]

Single Epithelial Cell Size--Class cov:
[[4.39618411 0.06127071]
 [0.06127071 0.02834291]]

Bare Nuclei--Class cov:
[[10.39260783  0.14929528]
 [ 0.14929528  0.02834291]]

Bland Chromatin--Class cov:
[[3.59737088 0.07334371]
 [0.07334371 0.02834291]]

Normal Nucleoli--Class cov:
[[7.94340086 0.10739084]
 [0.10739084 0.02834291]]

Mitoses--Class cov:
[[2.2551192  0.01750708]
 [0.01750708 0.02834291]]

